<a href="https://colab.research.google.com/github/jkellett11/sds510/blob/main/Essentials_MOD_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module 3 - Transformation Essentials

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import csv

In [26]:
#reading in the original crime dataset from the basics project, dropping the incomplete lines
crimedf = pd.read_csv('crimestatphx.csv', engine='python', on_bad_lines='skip')
crimedf.head(2)

,INC NUMBER,OCCURRED ON,OCCURRED TO,UCR CRIME CATEGORY,100 BLOCK ADDR,ZIP,PREMISE TYPE
0,201600000052855,11/01/2015 00:00,01/09/2016 00:00,MOTOR VEHICLE THEFT,N 43RD AVE & W CACTUS RD,85029.0,SINGLE FAMILY HOUSE
1,201600000594484,11/01/2015 00:00,NaN,RAPE,13XX E ALMERIA RD,85006.0,SINGLE FAMILY HOUSE


In [27]:
#converting the 'occurred on' and 'occurred to' columns to datetime
crimedf['OCCURRED ON'] = pd.to_datetime(crimedf['OCCURRED ON'])
crimedf['OCCURRED TO'] = pd.to_datetime(crimedf['OCCURRED TO'])
#finding the min and max date for the dataframe
mindate = crimedf['OCCURRED ON'].min()
maxdate = crimedf['OCCURRED ON'].max()
print(f'{mindate}, {maxdate}')

2015-11-01 00:00:00, 2017-11-17 05:50:00


In [28]:
#recreating the non-violent and violent category type to use for comparison later.
crimedf['CRIME TYPE'] = np.where(crimedf['UCR CRIME CATEGORY'].str.contains('ROBBERY|RAPE|MURDER AND NON-NEGLIGENT MANSLAUGHTER|AGGRAVATED ASSAULT'), 'VIOLENT', 'NONVIOLENT')
#checking mapping.
display(crimedf.head(2))

,INC NUMBER,OCCURRED ON,OCCURRED TO,UCR CRIME CATEGORY,100 BLOCK ADDR,ZIP,PREMISE TYPE,CRIME TYPE
0,201600000052855,2015-11-01,2016-01-09,MOTOR VEHICLE THEFT,N 43RD AVE & W CACTUS RD,85029.0,SINGLE FAMILY HOUSE,NONVIOLENT
1,201600000594484,2015-11-01,NaT,RAPE,13XX E ALMERIA RD,85006.0,SINGLE FAMILY HOUSE,VIOLENT


In [56]:
#extracting the year from the previously converted date column.
crimedf['YEAR'] = crimedf['OCCURRED ON'].dt.to_period('Y')
#getting a total count of crimes reported per year grouped by area.
yearcount = crimedf.groupby(['YEAR']).size().reset_index(name='Number of Crimes')
#GROWTH RATE (YEARLY) -- long term changes
#used pct_change to calculate the yearly change in the number of crimes year to year
yearcount['GrowthRate'] = yearcount['Number of Crimes'].pct_change()*100
display(yearcount)

,YEAR,Number of Crimes,GrowthRate
0,2015,10124,NaN
1,2016,63226,524.516002
2,2017,58010,-8.249771


Adapted from my previous code from the basics project. This calculation further proves the peak in crime data for 2016, with 524%. The rate could be seen as very high as there is no way to find the 2015 rate data as previous years were not included in the dataset. However, we can see that there was a small shrink rate seen in 2017 when the rate dropped to -8.24%.

In [33]:
#extracting the month and year for the bar plot
crimedf['year'] = crimedf['OCCURRED ON'].dt.year
crimedf['month'] = crimedf['OCCURRED ON'].dt.month
#grouping the month and year by the crime category
crimemonthlycounts = crimedf.groupby(['year','month'])['UCR CRIME CATEGORY'].size().reset_index(name='COUNT')
#grouping the total crimes by year to get a yearly breakdown
crimetypecounts = crimedf.groupby(['year','month','UCR CRIME CATEGORY']).size().reset_index(name='COUNT')
#creating a dataframe that groups the number of crimes by when they occurred, used to make a merged df for plotting -- may disregard.
crimetype = crimedf.groupby(['OCCURRED ON'])['UCR CRIME CATEGORY'].size().reset_index(name='COUNT')
crimetype.head(2)

,OCCURRED ON,COUNT
0,2015-11-01 00:00:00,10
1,2015-11-01 00:01:00,14


In [10]:
fig = px.bar(crimemonthlycounts, x='year', y='COUNT', color= crimemonthlycounts['year'].astype(str),
             title='Number of Total Crimes Per Year',
             template = 'plotly_dark')
fig.show()

The year 2016 saw the highest occurence of crime overall with it reaching 63,226 in total. Followed by 2017 with about 58,010 crimes total. It's important to note that the year 2015 is noticably low as the dataset only contains two months worth of data, with 10,124 occurrences of crime total.

In [11]:
fig = px.bar(crimemonthlycounts, x='month', y='COUNT', color= crimemonthlycounts['year'].astype(str),
             title='Number of Total Crimes Per Month For Each Year',
             template = 'plotly_dark')
fig.show()

When examining the crime trends over the years, we can see the total number of crimes occurring remains fairly consistent month to month apart from the month of December for 2017.

Next, I will examine the number of crimes per each crime, month to month for each year of the dataset. I opted to use a plotly graph as it allows me to examine each crime individually by selecting it in the legend.

In [12]:
fig = px.bar(crimetypecounts, x='month', y='COUNT', color= 'UCR CRIME CATEGORY',
             facet_col=crimetypecounts['year'].astype(str),
             title='Crime Total Count by Crime Per Year',
             template = 'plotly_dark')
fig.show()

Confirming my pervious findings in the basics project, the most common crimes revolve around theft. The most prominent being larceny-theft, followed by burglary, and then vehicle theft. Apart from crimes like murder, the appears to be a subtle increase in the number of crimes in the later half of the year than the start of it. This is especially true in categories like 'arson' with the peak of the data reaching 33 in June and 34 in December in 2016. While in 2017, 'arson' peaked in september with 40 followed by 39 in June. Additionally, for the murder category, in 2016 a great deal of crimes occurred in the last half of the year (77) versus the beginning half (58). This is further echoed in 2017 with latter having 78 vs 52. This tend proved to be true upon futher examination of the graphs for each category, especially for the crimes like MVT when the graph may look fairly constant upon first glance, starting with 2,795 versus 4,425 in 2016. I will revisit this graph when examining each crimes potential correlation to the average temperature that month.

**Weather Investigation**

Now, I will begin looking at temperature data from Phoenix during this time.

In [13]:
#reading in weather data from kaggle dataset
weatherdf = pd.read_csv('temperature.csv')
#selecting relevant columns as this dataset contains many different states.
azweather = weatherdf[['datetime','Phoenix']]
#filtering to match the timeframe found in the crime data csv.
azweathertime = azweather[(azweather['datetime'] >= '2015-11-01') & (azweather['datetime'] <= '2017-11-18')]
display(azweathertime.head(2))

,datetime,Phoenix
27012,2015-11-01 00:00:00,300.140000
27013,2015-11-01 01:00:00,298.804654


In [14]:
#converting the temperatures from Kelvin to Farhenheit.
azweathertime.loc[:, 'Phoenix'] = round((azweathertime['Phoenix'] -273.15) * 9/5 + 32, 2)
azweathertime = azweathertime.rename(columns={'Phoenix':'Phoenix (F)'})

#converting the 'datetime' column to be able to extract the month and year for future graphs.
azweathertime['datetime'] = pd.to_datetime(azweathertime['datetime'])
azweathertime['year'] = azweathertime['datetime'].dt.year
azweathertime['month'] = azweathertime['datetime'].dt.month
display(azweathertime.head(2))

,datetime,Phoenix (F),year,month
27012,2015-11-01 00:00:00,80.58,2015,11
27013,2015-11-01 01:00:00,78.18,2015,11


In [15]:
#finding the mean of the temperatures grouped  by month and year.
monthly_avg = azweathertime.groupby(['year','month'])['Phoenix (F)'].mean().round(2).reset_index()
monthly_avg.head(2)

,year,month,Phoenix (F)
0,2015,11,57.68
1,2015,12,49.65


In [16]:
#renaming the 'datetime' column to 'OCCURRED ON' to attempt to merge the df.
azweathertime=azweathertime.rename(columns={'datetime':'OCCURRED ON'})

#attempting to make a merged dataframe to try to plot comparisons. used 'inner' as that would join the two dfs on matching rows that contain the same time stamp.
#did not include the crime category in the merged df to ensure that no temperature data was lost.
merged_df = pd.merge(crimetype, azweathertime, on='OCCURRED ON', how='inner')
display(merged_df)

,OCCURRED ON,COUNT,Phoenix (F),year,month
0,2015-11-01 00:00:00,10,80.58,2015,11
1,2015-11-01 01:00:00,2,78.18,2015,11
2,2015-11-01 02:00:00,4,75.22,2015,11
3,2015-11-01 03:00:00,1,71.79,2015,11
4,2015-11-01 04:00:00,2,67.82,2015,11
...,...,...,...,...,...
15876,2017-11-17 01:00:00,1,82.60,2017,11
15877,2017-11-17 02:00:00,1,78.80,2017,11
15878,2017-11-17 03:00:00,1,73.60,2017,11
15879,2017-11-17 04:00:00,1,69.98,2017,11


In [34]:
#scatterplot of the average temperature in Phoenix from 2015 to 2017.
fig = px.scatter(monthly_avg, x='month', y='Phoenix (F)',
                 color=monthly_avg['year'].astype(str),
                 title='Average Temperature in Phoenix from Nov. 2015 to Nov. 2017',
                 template = 'plotly_dark')
fig.show()

Based on the temperature data, we can that the temperature in Phoenix is fairly consistent year to year. With a notable increase in the summer months, June - August, which tend to be the hottest months of the year with temperatures reaching above 90 degrees.

When comparing the two graphs above, we can potentially notice an interesting relationship. As previously stated the year with the highest amount of crime reported is 2016. And according to the temperature data, 2016 appears to be the hottest especially in the hotter months of June - August. While 2017, appears to be a bit hotter for longer. Beyond that, for crimes like MVT there appears to be an subtle increase in crime occurences in the latter half of the years, particularly in the summer months. Could heat potentially be a factor in the rate of certain crimes like MVT?

**Exploring Traffic Data**

Dataset was obtained from the phoenixopendata.org site.

In [36]:
#reading in phx traffic data
#IsNTMT stands for non motorized traffic
trafficdf = pd.read_csv('Street_Traffic_Volumes.csv')
trafficdf.head(5)

,X,Y,OBJECTID,ID,IsNTMT,STREET,BLOCKNUM,BLOCKDIR,CROSSSTREET,CROSSBLOCK,...,VOLUME,COUNTTYPE,DIRECTION,DESCRIPTION,DISTRICT,COUNTSTATION,BUDGETYR,COMMENTS,VERIFIEDON,AVG_SPEED
0,646180.302822,8.949711e+05,25947,13780,0,GRAND AVE,1200,N,ROOSEVELT ST,1500,...,6830,MID4,South Bound,Special,4,2420,2016,NaN,2015/09/10 15:08:13+00,37.0
1,646180.302822,8.949711e+05,25948,13780,0,GRAND AVE,1200,N,16TH AVE,1600,...,6971,MID4,South Bound,Special,4,2420,2016,NaN,2015/09/09 22:06:44+00,37.0
2,646226.129265,8.950183e+05,25949,13780,0,GRAND AVE,1200,N,ROOSEVELT ST,1500,...,6298,MID4,North Bound,Special,4,2420,2016,NaN,2015/09/10 15:08:13+00,37.0
3,646226.129265,8.950183e+05,25950,13780,0,GRAND AVE,1200,N,16TH AVE,1600,...,6476,MID4,North Bound,Special,4,2420,2016,NaN,2015/09/09 22:06:44+00,37.0
4,645643.030184,1.012856e+06,25951,13784,1,DOVE VALLEY RD,33000,N,16TH AVE,1580,...,2083,MID2,West Bound,Cut Thru Traffic,2,5483,2016,NaN,2015/09/14 20:16:20+00,46.0


In [37]:
#combining the columns as they broke up an address and coordinates.
trafficdf['Address'] = trafficdf[trafficdf.columns[5:8]].apply(
    lambda x: ' '.join(x.astype(str)), axis=1
)

trafficdf['Cross Address'] = trafficdf[trafficdf.columns[8:11]].apply(
    lambda x: ' '.join(x.astype(str)), axis=1
)

trafficdf['Coordinates'] = trafficdf[trafficdf.columns[0:2]].apply(
    lambda x: ' , '.join(x.astype(str)), axis=1
)
#dropping the original columns that I just combined.
trafficdf.drop(['STREET', 'BLOCKNUM', 'BLOCKDIR', 'CROSSSTREET', 'CROSSBLOCK', 'CROSSBLOCKDIR','X', 'Y'], axis=1, inplace=True)

In [38]:
trafficdf.head(2)

,OBJECTID,ID,IsNTMT,COUNTDATE,VOLUME,COUNTTYPE,DIRECTION,DESCRIPTION,DISTRICT,COUNTSTATION,BUDGETYR,COMMENTS,VERIFIEDON,AVG_SPEED,Address,Cross Address,Coordinates
0,25947,13780,0,2015/09/01 07:00:00+00,6830,MID4,South Bound,Special,4,2420,2016,NaN,2015/09/10 15:08:13+00,37.0,GRAND AVE 1200 N,ROOSEVELT ST 1500 N,"646180.302821521 , 894971.058398947"
1,25948,13780,0,2015/09/02 07:00:00+00,6971,MID4,South Bound,Special,4,2420,2016,NaN,2015/09/09 22:06:44+00,37.0,GRAND AVE 1200 N,16TH AVE 1600 N,"646180.302821521 , 894971.058398947"


In [39]:
#dropped counttype and comments as they are not needed/empty
trafficdf.drop(['COUNTTYPE', 'COMMENTS'], axis=1, inplace=True)
#trafficdf['X'].dtype

In [40]:
#converting the countdate column to datetime
trafficdf['COUNTDATE'] = pd.to_datetime(trafficdf['COUNTDATE'])
#dropping the time zone from the countdate column
trafficdf['COUNTDATE'] = trafficdf['COUNTDATE'].dt.tz_convert(None)
#finding the min/starting date of the countdate column
trafficdf['COUNTDATE'].min()

Timestamp('2015-09-01 07:00:00')

In [43]:
#creating a dataframe that only returns rows that match the start and end date from the crime dataframe for comparison reasons.
trafficfilter = trafficdf[(trafficdf['COUNTDATE'] >= mindate) & (trafficdf['COUNTDATE'] <= maxdate)].copy()
trafficfilter.head(2)

,OBJECTID,ID,IsNTMT,COUNTDATE,VOLUME,DIRECTION,DESCRIPTION,DISTRICT,COUNTSTATION,BUDGETYR,VERIFIEDON,AVG_SPEED,Address,Cross Address,Coordinates
58,26121,13912,1,2015-11-03 07:00:00,884,South Bound,Cut Thru Traffic,8,1480,2016,2015/11/10 20:54:18+00,30.0,12TH ST 1200 E,CHAMBERS ST 5400 S,"657360.542650919 , 872177.060695536"
59,26122,13912,1,2015-11-03 07:00:00,998,North Bound,Cut Thru Traffic,8,1480,2016,2015/11/10 20:54:18+00,30.0,12TH ST 1200 E,BOWKER ST 5500 S,"657418.543963253 , 872187.898293965"


In [44]:
#extracting the month and year from 'countdate' for plotting
trafficfilter['year'] = trafficfilter['COUNTDATE'].dt.year
trafficfilter['month'] = trafficfilter['COUNTDATE'].dt.month
#creating trafficcounts to plot the volume of traffic seen each month per year.
#needed to added '.copy()' to avoid a 'settingwithcopywarning' error
trafficcounts = trafficfilter.groupby(['month', 'year'])['VOLUME'].sum().reset_index(name='COUNT')

In [46]:
fig = px.bar(trafficcounts, x='month', y='COUNT', color=trafficcounts['year'].astype(str),
             title='Total Traffic Per Month For Each Year',
             template = 'plotly_dark')
fig.show()

Here, we can see that for 2016, there particularily high pockets of traffic in the last half of the year starting in June to November. However, in 2017 the volume was a bit more spread out with the highest months for traffic being January and February, with 697k and 771k, followed by July with 687k. Similarly, to the crime dataframe, we can see that November was the peak for traffic data that year with 117k. This is primarily due to the trameframe we are examining/comparing to.

Traffic Growth Rate

In [59]:
#extracting the year from the previously converted date column.
trafficfilter['MONTH'] = trafficfilter['COUNTDATE'].dt.to_period('M')
#getting a total traffic volume reported per month.
monthcountvol = trafficfilter.groupby('MONTH')['VOLUME'].sum().reset_index(name='TOTAL TRAFFIC VOLUME')
#monthly growth rate for traffic volume
monthcountvol['GrowthRate'] = monthcountvol['TOTAL TRAFFIC VOLUME'].pct_change() * 100
display(monthcountvol)

,MONTH,TOTAL TRAFFIC VOLUME,GrowthRate
0,2015-11,117115,NaN
1,2015-12,2882,-97.539171
2,2016-01,64255,2129.528105
3,2016-02,497331,673.995798
4,2016-03,7734,-98.444899
5,2016-04,184128,2280.760279
6,2016-05,97043,-47.295903
7,2016-06,17493,-81.973970
8,2016-07,1023169,5749.019608
9,2016-08,789804,-22.808060


In [55]:
#extracting the year from the previously converted date column.
trafficfilter['YEAR'] = trafficfilter['COUNTDATE'].dt.to_period('Y')
#getting a total traffic volume reported per year.
yearcountvol = trafficfilter.groupby('YEAR')['VOLUME'].sum().reset_index(name='TOTAL TRAFFIC VOLUME')
#display(yearcountvol)

yearcountvol['GrowthRate'] = yearcountvol['TOTAL TRAFFIC VOLUME'].pct_change() * 100
display(yearcountvol)

,YEAR,TOTAL TRAFFIC VOLUME,GrowthRate
0,2015,119997,NaN
1,2016,5286726,4305.715143
2,2017,3678969,-30.411203


Here, I wished to examine the growth rate of traffic. Simiarly to the previous calculation for the crime growth rate, we can begin to see an interesting trend. In 2016, the growth rate for traffic volume peaked with 4305.71%. Followed by a decrease / shrink rate of -30.41% in 2017. While traffic rates can fluccuate as there could be many factors contributing to higher volumes one day to the next, it is interesting to see that 2016 was the peak for traffic similar to the crime dataset. Perhaps if the crime dataset contained data from a larger timeframe we could investigate this potential relationship more. Especially as this information could prove fuitful to law enforcement who wish to tamp down the crime rates as well as help with the traffic volume level. While traffic volume is not immediately an indication of high crimes rates, it does depict areas that are highly trafficed. Additionally, when examining the monthly growth rate of traffic we can see that in July of 2016 rates peaked with 5749.01% followed by a peak in June of the next year with 688.79%. In addition, these findings coupled with the temperature data that was previously explored, we can see an emerging trend of crime occurences peaking in the last half of the month as well as traffic volumes reaching an all time high as well. However, it is important to note that whether crimes are peaking at the same time is heavily dependent on the crime category.

**Poverty Level Exploration**

Here, I will begin examining the poverty level in phoenix from 2015 - 2017. Dataset was obtained from the phoenixopendata.org site.

In [41]:
#since this dataset is so large, I will focus primarily on the total values of income below and at poverty level for each year.
povertydf = pd.read_csv('poverty.csv')
povertydf = povertydf[(povertydf['year'] >= 2015) & (povertydf['year'] <= 2017)]
povertydf = povertydf.drop(columns={'_id', 'geoid'})
povertydf

,year,Total Population 5 years and over,Income below poverty level,Income below poverty level - Male,Income below poverty level - Male - Under 5 years,Income below poverty level - Male - 5 years,Income below poverty level - Male - 6 to 11 years,Income below poverty level - Male - 12 to 14 years,Income below poverty level - Male - 15 years,Income below poverty level - Male - 16 and 17 years,...,Income at or above poverty level - Female - 12 to 14 years,Income at or above poverty level - Female - 15 years,Income at or above poverty level - Female - 16 and 17 years,Income at or above poverty level - Female - 18 to 24 years,Income at or above poverty level - Female - 25 to 34 years,Income at or above poverty level - Female - 35 to 44 years,Income at or above poverty level - Female - 45 to 54 years,Income at or above poverty level - Female - 55 to 64 years,Income at or above poverty level - Female - 65 to 74 years,Income at or above poverty level - Female - 75 years and over
7,2015,1545360,344455,158414,18302,2260,25596,10948,3197,5955,...,20215,8037,15811,52901,90230,82798,83541,70015,45647,33006
8,2016,1595696,324498,146953,18042,3055,23449,8556,4069,6424,...,23128,6814,17653,56468,97178,87344,88848,76059,47562,31457
9,2017,1604181,268855,125780,13414,1600,18686,9615,2571,5829,...,29103,8438,15798,62918,105651,91055,85953,77420,50259,34971


In [42]:
#plotting the number of the population's income that is below the poverty level.
fig1 = px.bar(povertydf, x='Income below poverty level', color=povertydf['year'].astype(str),

             title='Poverty Levels (Below) Per Year', template = 'plotly_dark')
fig1.show()

#plotting the number of the population's income that is at or above the poverty level.
fig2 = px.bar(povertydf, x='Income at or above poverty level', color=povertydf['year'].astype(str),

             title='Poverty Levels (At or Above) Per Year', template = 'plotly_dark')
fig2.show()

In [43]:
#plotting the number of the population's income that is below the poverty level.
fig1 = px.bar(povertydf, x='Income below poverty level - Male', color=povertydf['year'].astype(str),

             title='Poverty Levels (Below) Per Year', template = 'plotly_dark')
fig1.show()

#plotting the number of the population's income that below the poverty level.
fig2 = px.bar(povertydf, x='Income below poverty level - Female', color=povertydf['year'].astype(str),

             title='Poverty Levels (Below) Per Year', template = 'plotly_dark')
fig2.show()

According to the bar plots above, in the three year span, there were a considerable portion of the population that were at or above poverty level. Perhaps if this data contained specific areas such as zip code, we could could cross reference the crime rates in the area by zip code, which I started in the basics project and potentially see if there what relationship poverty has to this crime dataset. Especially as many often think that high areas poverty usually contain higher crime rates due to the possibility of lack of employment or opportunity. While that notion may be true thus far, we could get at the heart of whether that rings true for phoenix and for which crimes if it does. Also, one interesting trend I noticed in this dataset was that females consistently were found to be a higher percentage of the population that were below the poverty level.

Potential ideas for future expansion on the topic:
- examining poverty rates in deeper depth by zipcode.
- examining richer traffic data, perhaps exploring road rage incidents as temperature could be thought to play a factor in these incidents occurring.
- housing data of the area, this could potentially tie in with poverty levels as well. (I initally tried to include a housing dataset of Phoenix but struggled to find one matching this specific timeframe.)

As stated previously, crime data is vital for not only to law enforcement but to potential homeowners as well. But deeper than this, is how these crime rates are affected by other confounding variable like weather, traffic and poverty level--especially to those in law enforcement trying to curb increasing crime rates. But also to those wishing to set down roots by moving into a certain area with their family.